In [1]:
import astropy as ap
import astropy.io.fits as ap_fits
import astropy.modeling as ap_mod
import copy
import glob
import inspect
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as np_ma
import pandas as pd
import scipy as sp
import scipy.optimize as sp_opt
import scipy.stats as sp_stat
import warnings as warn

import IfA_Smeargle as ifas

In [4]:
first_run = ifas.yankee.SmeargleConfig('FirstRun.ifaspkl')

first_run.BravoConfig.voltpat_rename_config['archive_data'] = False
first_run.BravoConfig.number_rename_config['archive_data'] = False
first_run.BravoConfig.set_determine_rename_config['archive_data'] = False
first_run.BravoConfig.detector_name = {'name': 'SAPHIRA-MK20-M12145-03_20190927_T40K'}

if (False):
    first_run.BravoConfig.voltpat_rename_config['voltage_pattern'] =  [2.5, 2.5, 2.5, 2.5,
                                                                 4.5, 4.5, 4.5, 4.5,
                                                                 6.5, 6.5, 6.5, 6.5,
                                                                 8.5, 8.5, 8.5, 8.5,
                                                                 9.5, 9.5, 9.5, 9.5,
                                                                 10.5, 10.5, 10.5, 10.5,
                                                                 11.5, 11.5, 11.5, 11.5,
                                                                 11.5, 11.5, 11.5, 11.5,
                                                                 10.5, 10.5, 10.5, 10.5,
                                                                 9.5, 9.5, 9.5, 9.5,
                                                                 8.5, 8.5, 8.5, 8.5,
                                                                 6.5, 6.5, 6.5, 6.5,
                                                                 4.5, 4.5, 4.5, 4.5,
                                                                 2.5, 2.5, 2.5, 2.5]
    first_run.BravoConfig.voltpat_rename_config['begin_garbage'] = 0

print(first_run)


< Configuration Class --- SmeargleConfig > 
╓ SmeargleConfig Parameters:
╟  BravoConfig ↴ 
< Configuration Class --- BravoConfig > 
╓ BravoConfig Parameters:
╟  detector_name ⇀ {'name': 'SAPHIRA-MK20-M12145-03_20190927_T40K'}
╟  same_file_size_sanitization_config ⇀ {'method': None}
╟  number_rename_config ⇀ {'begin_garbage': 1, 'archive_data': False}
╟  set_determine_rename_config ⇀ {'set_length': 14, 'begin_garbage': 1, 'archive_data': False}
╟  voltpat_rename_config ⇀ {'voltage_pattern': [2.5, 4.5, 6.5, 8.5, 9.5, 10.5, 11.5, 11.5, 10.5, 9.5, 8.5, 6.5, 4.5, 2.5], 'begin_garbage': 1, 'archive_data': False}
╟  auto_avg_slicing_config ⇀ {'reference_chunk': None, 'split_chunks': None, 'averaging_function': None}
╟  avg_endpoints_config ⇀ {'start_chunk': None, 'end_chunk': None}
╟  avg_endpts_persec_config ⇀ {'start_chunk': None, 'end_chunk': None, 'frame_exposure_time': None}
╟  avg_endpts_perksec_config ⇀ {'start_chunk': None, 'end_chunk': None, 'frame_exposure_time': None}
╙-----------

In [3]:
ifas.zulu.pipelines.SA201907281826_reduction_pipeline('./Data',first_run)

In [ ]:
first_run.ZuluConfig.SA201907281826 = {'early_frame': 3, 'ref_frame': np.array([161,192]), 'sub_avg_frames': np.array([[221,228]]), 'frame_exposure': 5, 'early_sigma': 5}
print(first_run)
ifas.zulu.pipelines.SA201907281826_reduction_pipeline('./Data',first_run)

In [ ]:
data
print(np.nanmean(data.data),np.nanstd(data.data))

In [ ]:
first_run.OscarConfig.general_histogram_config['histogram_plot_paramters']['range'] = [-1000,1000]

figure = ifas.oscar.multi.plot_single_heatmap_and_histogram("./Data\Saphria__num;14__detBias;4.5Vdown__1025-1040.fits",first_run)
#figure.axes[1].axvline(np.nanmean(data.data))
plt.show(figure)

In [ ]:
first_run.OscarConfig.general_histogram_config['fit_gaussian'] = True
#first_run.OscarConfig.general_histogram_config['bin_width'] = 5
#first_run.OscarConfig.general_histogram_config['histogram_plot_paramters']['range'] = [-250,100]

__, __, data = ifas.meta.meta_faa.smeargle_open_fits_file("./Data/SAPHIRA-MK12145-02_20190913-40K__num;7__detBias;10.5Vup__slice;513-528.fits")
figure2 = ifas.oscar.multi.plot_single_heatmap_and_histogram(data,first_run)
#figure3 = ifas.oscar.multi.plot_single_heatmap_and_histogram(data.data,first_run)
#figure.axes[1].axvline(np.nanmean(data.data))
plt.show(figure2)
plt.show(figure3)

# [ 72 120 136 250 294]
#[-16074. -14058. -13386.  -8598.  -6750.]
ifas.meta.meta_plting.smeargle_save_figure_file(figure2,"website.png")
print(np.histogram(data.data,bins=10))

In [ ]:
__, __, data = ifas.meta.meta_faa.smeargle_open_fits_file("./Data\Saphria__num;14__detBias;4.5Vdown__257-272.fits")
figure2 = ifas.oscar.histograms.plot_array_histogram(data.data,bin_width=10,fit_gaussian=True)
#figure.axes[1].axvline(np.nanmean(data.data))
plt.show(figure2)

wee, weer = ifas.meta.meta_model.smeargle_fit_histogram_gaussian_function(data.data)

print(np.nanstd(data))

In [ ]:
# Extract histogram data from the data.
hist_data = np.histogram(data.data, bins=ifa.oscar.oscar_bin_width(data.data, 10))
hist_x = (hist_data[1][0:-1] + hist_data[1][1:]) / 2 # Middle of bin.
hist_y = hist_data[0]
   # Filter out some of the outlier pixels, consider only 75% of the 
    # meaningful bins and the bins with a value greater than a limiting entry.
valuecut_index = np.where(hist_y >= 11)
cuthist_x = np.array(hist_x[valuecut_index])
cuthist_y = np.array(hist_y[valuecut_index])

    # Initial guesses...
    # The peak of the data is a good guess for the mean value.
guess_mean = hist_x[np.argmax(cuthist_y)]
print(np.nanstd(cuthist_y))
print(sp_stat.median_absolute_deviation(cuthist_y))
print(sp_stat.variation(cuthist_y))

In [ ]:
ifas.bravo.sanitize.same_file_size_sanitization('./Data')

In [ ]:
glob.glob('./Data*.fits')

In [ ]:
fig, ax = plt.subplots(dpi=300)
ax, data_table, data_frame = ifas.oscar.volt_plot.plotdir_dark_current_over_voltage('./Data/', figure_axes=ax)

#ax.set_yscale('log')
ax.set_ylim((-250,250))

plt.show()

In [ ]:
data_table

In [ ]:
fig2, ax2 = plt.subplots(dpi=300)

ax2.errorbar(data_table['voltage_set1'][0:5], data_table['value_set1'][0:5], yerr=data_table['error_set1'][0:5],
                    fmt='.-', elinewidth=0.25, capsize=3, capthick=0.25,
                    label=('Set ' + str(1)))
ax2.errorbar(data_table['voltage_set2'][0:5], data_table['value_set2'][0:5], yerr=data_table['error_set2'][0:5],
                    fmt='.-', elinewidth=0.25, capsize=3, capthick=0.25,
                    label=('Set ' + str(2)))

title=first_run.BravoConfig.detector_name['name']
ax2.set_title(title)
ax2.legend(loc='upper left')
ax2.set_xticks(np.append(data_table['voltage_set1'][0:5], 0))
ax2.set_xlabel('Detector Bias Voltage (V)')
ax2.set_ylabel('Average Dark Current (ADU)')

plt.show()


In [ ]:
np.append(data_table['voltage_set1'][0:5], 0)

In [ ]:
dataframe.query('volt == @voltage_value and voltslope == @voltage_slope')['num']

In [ ]:
voltage_list = copy.deepcopy(np.sort(np.unique(file_data.loc[:,'volt'].to_numpy())))

set_metadata = []
#slope_list = list(set(dataframe.query('volt == @voltage_value')['voltslope']))

for voltdex in voltage_list:
    slope_list = list(set(file_data.query('volt == @voltdex')['voltslope']))
    
    for slopedex in slope_list:
        query = file_data.query('volt == @voltdex and voltslope == @slopedex')['num']
        filenum = np.sort(np.unique(query))
        
        for numdex, setdex in zip(filenum,np.arange(len(filenum))+1):
            temp_dict = {'metavolt':voltdex,'metaslope':slopedex,'metanum':numdex,'metaset':setdex}
            set_metadata.append(temp_dict)
            
set_metadata = pd.DataFrame(set_metadata)

In [ ]:
set_count = np.flip(np.unique(set_metadata['metaset']))[np.flip(np.unique(set_metadata['metaset'],return_counts=True)[1]).argmax()]

for setdex in (np.arange(set_count) + 1):
    dataset_list = set_metadata.query('metaset == @setdex')
    
    x_axis_voltage = []
    y_axis_data = []
    y_axis_error = []
    
    for voltdex in voltage_list:
        fits_file_numbers = list(dataset_list.query('metavolt == @voltdex')['metanum'].to_numpy())
        voltset_data = file_data.query('num == @fits_file_numbers')
        
        x_axis_voltage.append(voltdex)
        y_axis_data.append(np.nanmedian(voltset_data['g_mean']))
        y_axis_error.append(np.nanmedian(voltset_data['g_stddev']))
        
    #####
    x_axis_voltage = np.array(x_axis_voltage)
    y_axis_data = np.array(y_axis_data)
    y_axis_error = np.array(y_axis_error)
    plt.errorbar(x_axis_voltage, y_axis_data, yerr=y_axis_error)


In [ ]:
import importlib

complete_oscar_dict = dict(inspect.getmembers(ifas.oscar, inspect.ismodule))

source_file_list = [inspect.getsourcefile(complete_oscar_dict[keydex]) for keydex in complete_oscar_dict.keys()]
ifa_smeargle_file_list = [filedex for filedex in source_file_list if 'IfA_Smeargle' in filedex]
oscar_file_list = [filedex for filedex in ifa_smeargle_file_list if 'oscar' in filedex]

oscar_mod_name_list = [inspect.getmodulename(oscardex) for oscardex in ifa_smeargle_file_list if 'oscar' in oscardex]

oscar_modspec = [importlib.util.spec_from_file_location(namedex, filedex) for namedex, filedex in zip(oscar_mod_name_list, oscar_file_list)]
oscar_modules = [importlib.util.module_from_spec(specdex) for specdex in oscar_modspec]

for specdex, moduledex in zip(specifications,oscar_modules):
    specdex.loader.exec_module(moduledex)

function_dictionaries = [dict(inspect.getmembers(moduledex,inspect.isfunction)) for moduledex in oscar_modules]

oscar_plotting_functions = {}
for dictionarydex in function_dictionaries: 
    oscar_plotting_functions = {**oscar_plotting_functions, **dictionarydex}
for keydex, valuedex in copy.deepcopy(oscar_plotting_functions).items():
    if ('smeargle' in keydex):
        oscar_plotting_functions.pop(keydex, None)
    
print(oscar_plotting_functions)

In [ ]:
data_array = ifas.zulu.IfasDataArray('./Data/SAPHIRA-MK20-M12145-03_20191001_T60K__num;2__detBias;2.5Vup__slice;1025-1040.fits', 
                                     configuration_class='FirstRun.ifaspkl')

In [ ]:
print(data_array.__doc__)

In [ ]:
data_array.config.EchoConfig.echo271_config['maximum_value'] = -50
print(data_array.config)

In [ ]:
data_array.echo271_maximum_cut(run=False, configuration=True)

In [ ]:
__, __, ifas_data = ifas.meta.meta_faa.smeargle_open_fits_file('./Data/' + 
                        'SAPHIRA-MK20-M12145-03_20190927_T40K__num;14__set;1__detBias;2.5Vdown__slice;221-228'
                                                               +'.fits')
__, __, shane_data = ifas.meta.meta_faa.smeargle_open_fits_file('/Users/Tripp/Downloads/190927_T40K_2hrdark_set1/' +
                        '190927_T40K_2hrdark_2.5VdetBias_down_16avg_CDS'
                                                                + '.fits')
if (False):
    magnitude = np.ma.divide(ifas_data, shane_data)
    inv_mag = np.ma.divide(1, magnitude)
else:
    magnitude = (ifas_data / shane_data)
    inv_mag = (1 / magnitude)

plt.hist(inv_mag.flatten(), bins=int(inv_mag.flatten().ptp()), log=True)
plt.xlim([-100,100])
plt.show()

plt.imshow(inv_mag)
plt.colorbar()
plt.show()

print('Mean   ' + str(np.nanmean(inv_mag)))
print('Median   ' + str(np.ma.median(inv_mag)))
print('Std   ' + str(np.std(inv_mag)))
print('Var   ' + str(np.var(inv_mag)))

print('x̄' + str(round(np.nanmean(inv_mag),2)) + ', μ' + str(round(np.ma.median(inv_mag),2)) + ', σ' 
      + str(round(np.std(inv_mag),2)) + ', σ*' + str(round(np.var(inv_mag),2)))

In [ ]:
ifas.meta.meta_math.smeargle_mean(ifas_data)

In [ ]:
first_run.OscarConfig.general_histogram_config['histogram_plot_paramters']['range'] = [-100,100]

figure = ifas.oscar.multi.plot_single_heatmap_and_histogram(shane_data,first_run)
#figure.axes[1].axvline(np.nanmean(data.data))
plt.show(figure)

In [ ]:
__, __, ifas_data = ifas.meta.meta_faa.smeargle_open_fits_file('./Data/' + 
                        'SAPHIRA-MK20-M12145-03_20190927_T40K__num;13__set;1__detBias;4.5Vdown__slice;221-228'
                                                               +'.fits')
ifas_data

In [ ]:
__, __, true_data = ifas.meta.meta_faa.smeargle_open_fits_file('/Users/Tripp/Desktop/RawData/MK20-M12145-03_20190927_T40K/' + 
                        '190928_203649'
                                                               +'.fits')

In [ ]:
first_chunk = true_data[161:192,:,:]
second_chunk = true_data[221:228,:,:]

first_chunk_median = np.nanmedian(first_chunk, axis=0)
second_chunk_median = np.nanmedian(second_chunk, axis=0)

diff = second_chunk_median - first_chunk_median

In [5]:
test = ifas.zulu.IfasDataArray(None, blank=True)

C:\Users\Tripp\Desktop\PROGRAM\IfA_Smeargle\zulu\zulu_main.py:79: InputWarning: The data array file is indicated to return a blank one. Doing so as requested.
  smeargle_warning(InputWarning,("The data array file is indicated to return a blank "
C:\Users\Tripp\Desktop\PROGRAM\IfA_Smeargle\zulu\zulu_main.py:102: InputWarning: There is no configuration class provided for this array. The attribute will be None. The SmargleConfig class will be the default.
  smeargle_warning(InputWarning,("There is no configuration class provided for this "
C:\Users\Tripp\Desktop\PROGRAM\IfA_Smeargle\zulu\zulu_main.py:343: InputWarning: There is no input configuration class. A blank one will be assigned.
  smeargle_warning(InputWarning,("There is no input configuration class. A blank one "


In [6]:
[method_name for method_name in dir(test)
                  if callable(getattr(test, method_name))]

['__class__',
 '__delattr__',
 '__dir__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_bravo_functionality',
 '_echo_functionality',
 '_oscar_functionality',
 '_yankee_functionality',
 '_zulu_determine_configuration_parameters',
 'bravo_filename_split_by_parameter',
 'echo010_fixing_invalids',
 'echo120_subarray_mask',
 'echo170_gaussian_truncation',
 'echo270_minimum_cut',
 'echo271_maximum_cut',
 'echo275_pixel_truncation',
 'echo276_percent_truncation',
 'echo277_sigma_truncation',
 'echo380_single_pixels',
 'echo381_rectangle_mask',
 'echo382_column_mask',
 'echo383_row_mask',
 'echo398_nothing',
 'echo399_everything',
 'echo_create_masked_array',
 'echo_execution',
 'echo_synthesize_mask_dictionary',
 'fast_forward_configuration',
 'get_data',
 'median_en

In [ ]:
test = ifas.zulu.IfasDataArray('./Data/191103_001938.fits', blank=False)

In [ ]:
test.median_endpoints(start_chunk=[161,192], end_chunk=[221,228])

In [ ]:
test.plot_array_heatmap_image()

In [ ]:
test.echo170_gaussian_truncation(sigma_multiple=5,bin_size=100)

In [ ]:
test.echo_synthesize_mask_dictionary()
test.echo_create_masked_array()

In [ ]:
test.plot_single_heatmap_and_histogram(bin_width=10)

In [ ]:
test.get_data()

In [ ]:
# Just a inner function to store the new naming routine. 
# Be adaptive as to which configuration class is given.
provided_config = yankee.yankee_extract_proper_configuration_class(configuration_class,
                                                                   yankee.BravoConfig)
# Determine the detector's name.
if (isinstance(provided_config.detector_name,str)):
    detector_name = provided_config.detector_name
elif (isinstance(provided_config.detector_name,dict)):
    try:
        detector_name = provided_config.detector_name['name']
    except KeyError:
        raise ConfigurationError("The detector name cannot be found in the "
                                 "configuration file. It must either be a string or a "
                                 "dictionary with the name value referenced by the key "
                                 "<name>.")
else:
    raise ConfigurationError("The detector name cannot be found in the configuration "
                             "file. It must either be a string or a dictionary with "
                             "the name value referenced by the key <name>.")

# Getting the voltage renames
number_names = bravo.rename.number_renaming(data_directory,
                                            **provided_config.number_rename_config)
set_names = bravo.rename.set_determinization_renaming(data_directory,
                                                      **provided_config.set_determine_rename_config)
voltage_names = bravo.rename.voltage_pattern_renaming(data_directory,
                                                      **provided_config.voltpat_rename_config)
n_files = len(number_names)


# Rename all of the files. The directory structure seems fine.
final_names = []
for numdex, setdex, detectdex, voltdex in zip(number_names,
                                              set_names,
                                              [detector_name for index in range(n_files)],
                                              voltage_names):
final_names.append(detectdex
                   + '__' + numdex
                   + '__' + setdex
                   + '__' + voltdex 
                   # + '__'
                   + '.fits')

bravo.bravo_rename_parallel(None, final_names, data_directory, file_extensions='.fits')